<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">INFO</h1>

In this script, we create a class that will structure the unit tests for the `Dijkstra` player. \
We choose to use the `unittest` library. \
Then, we run them to ensure that all methods developed work as expected.

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">IMPORTS</h1>

In [4]:
# External imports
from typing import *
from typing_extensions import *
from numbers import *
import unittest
import sys
import os
import random

# Add needed directories to the path
sys.path.append(os.path.join("..", "players"))

# PyRat imports
from Dijikstra import Dijikstra
from pyrat import BigHolesRandomMaze, Action

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">DEFINE THE TESTS</h1>

The `unittest` library requires the creation of a class that extends `unittest.TestCase`. \
For each method to test, we need to define a method in the test class. \
Each of these test methods should call the tested method with various inputs to check that produced outputs match expected ones.

In [5]:
class DFSTests (unittest.TestCase):

    """
        This class tests the methods of the DFS class.
        For each method, we test it with a few different configurations.
    """

    #############################################################################################################################################
    #                                                                 UNIT TESTS                                                                #
    #############################################################################################################################################

    def test_traversal ( self: Self
                       ) ->    None:

        """
            This method tests the 'traversal' method.
            We are going to check the following:
            - Outputs are of the expected types.
            - All vertices are visited.
            - The routing table is a tree, with root corresponding to the start of the traversal.
            - The found distances are correct, i.e., strictly positive and increasing, except for the start vertex which should be 0.
            Note that we cannot test that the output is unique, as there are multiple valid outputs, depending on how vertices are explored.
            We will test the method on several random graphs to make sure it is working properly.
            Random graphs will be generated using the PyRat class used to create mazes in games.
            There are several such classes, but we will use the BigHolesRandomMaze class.
            In:
                * self: Reference to the current object.
            Out:
                * None.
        """

        # Constants
        NB_GRAPHS = 10
        WIDTHS = [2, 30]
        HEIGHTS = [2, 30]
        CELL_PERCENTAGES = [20.0, 100.0]
        WALL_PERCENTAGES = [20.0, 100.0]
        MUD_PERCENTAGE = 0.0

        # Test on several graphs
        for i in range(NB_GRAPHS):
            
            # Instantiate the player
            player = Dijikstra()

            # Generate a random maze
            # We use a fixed random seed for reproducibility of tests
            rng = random.Random(i)
            maze = BigHolesRandomMaze(width = rng.randint(WIDTHS[0], WIDTHS[1]),
                                      height = rng.randint(HEIGHTS[0], HEIGHTS[1]),
                                      cell_percentage = rng.uniform(CELL_PERCENTAGES[0], CELL_PERCENTAGES[1]),
                                      wall_percentage = rng.uniform(WALL_PERCENTAGES[0], WALL_PERCENTAGES[1]),
                                      mud_percentage = MUD_PERCENTAGE,
                                      random_seed = i)

            # Choose a random starting vertex
            start_vertex = rng.choice(maze.vertices)

            # Perform the traversal
            distances, routing_table = player.traversal(maze, start_vertex)

            # Check the output type for distances
            # It should be a dictionary with integer keys and integer values
            self.assertTrue(isinstance(distances, Dict))
            self.assertTrue(all(isinstance(k, Integral) for k in distances.keys()))
            self.assertTrue(all(isinstance(v, Integral) for v in distances.values()))
            
            # Check the output type for the routing table
            # It should be a dictionary with integer keys and integer or None values
            self.assertTrue(isinstance(routing_table, Dict))
            self.assertTrue(all(isinstance(k, Integral) for k in routing_table.keys()))
            self.assertTrue(all(isinstance(v, (type(None), Integral)) for v in routing_table.values()))

            # All vertices should be visited
            # Equivalently, the distances should have the same keys as the maze vertices
            self.assertEqual(sorted(set(distances.keys())), sorted(maze.vertices))

            # Check that the start vertex is the only tree root
            self.assertEqual(routing_table[start_vertex], None)
            self.assertTrue(all(routing_table[v] is not None for v in routing_table.keys() if v != start_vertex))
            self.assertEqual(distances[start_vertex], 0)
            self.assertTrue(all(distances[v] > 0 for v in distances.keys() if v != start_vertex))
            
            # We check that the routing table is a tree
            # Also we check that associated distances are decreasing as we go to the root
            for vertex in routing_table:

                # We check that the parent is closer to the root
                if routing_table[vertex] is not None:
                    self.assertTrue(distances[routing_table[vertex]] < distances[vertex])
                
                # We ckeck that we can reach the root from any vertex
                path = [vertex]
                while routing_table[path[-1]] is not None:
                    self.assertTrue(routing_table[path[-1]] not in path)
                    path.append(routing_table[path[-1]])
    def test_find_route ( self: Self,
                    ) ->    None:

        # Constants
        INPUTS = [{"routing_table": {0: None, 1: 0, 2: 1, 3: 2, 4: 3},
                "start": 0,
                "end": 4},
                {"routing_table": {0: 2, 1: None, 2: 1, 3: 0, 4: 0, 5: 4, 6: 1},
                "start": 1,
                "end": 6}]

        # Test on several controlled examples
        for i in range(len(INPUTS)):
            
            # Instantiate the player
            player = Dijikstra()

            # Get the input data
            routing_table = INPUTS[i]["routing_table"]
            start = INPUTS[i]["start"]
            end = INPUTS[i]["end"]

            # Find the route
            route = player.find_route(routing_table, start, end)

            # Check the output type
            # It should be a list of integers
            self.assertTrue(isinstance(route, List))
            self.assertTrue(all(isinstance(v, Integral) for v in route))

            # Check that the route is a valid path from the start to the end
            self.assertEqual(route[0], start)
            self.assertEqual(route[-1], end)
            self.assertTrue(all(routing_table[route[j + 1]] == route[j] for j in range(len(route) - 1)))

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">RUN THE TESTS</h1>
           
When calling `unittest.main()`, all methods in the test class above will be run.

In [6]:
# Run all tests
_ = unittest.main(argv=[""], verbosity=2, exit=False)

test_find_route (__main__.DFSTests) ... ok
test_traversal (__main__.DFSTests)
This method tests the 'traversal' method. ... 

Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor


ok

----------------------------------------------------------------------
Ran 2 tests in 3.015s

OK
